# AHL Player API Examples

This notebook demonstrates how to use the AHL player scrapers to fetch:
- Player biographical information
- Season statistics
- Career statistics
- Game-by-game logs
- Shot location data
- Draft information

In [ ]:
import pandas as pd
import sys
sys.path.append('..')

from scrapernhl.ahl.scrapers import (
    get_player_profile,
    get_player_bio,
    get_player_stats,
    get_player_game_log,
    get_player_shot_locations,
    get_player_draft_info,
    scrape_player_profile,
    scrape_player_stats,
    scrape_player_career_stats,
    scrape_player_game_log,
    scrape_player_shot_locations,
    scrape_multiple_players
)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 1. Get Player Profile (Raw Data)

Get the complete player profile including all available data.

In [ ]:
# Example: David Reinbacher (player_id = 10036)
player_id = 10036

profile = get_player_profile(player_id)

# Show the structure of the response
print("Available keys in profile:")
print(profile.keys())

# Show profile as JSON
import json
print("\nFull profile:")
print(json.dumps(profile, indent=2)[:2000])  # First 2000 chars

## 2. Get Player Biographical Info

In [ ]:
# Get just the biographical information
bio = get_player_bio(player_id)

print("Player Biographical Info:")
print(json.dumps(bio, indent=2))

## 3. Get Player Season Statistics

In [ ]:
# Get player season and career stats
stats = get_player_stats(player_id)

print("Season Stats:")
print(json.dumps(stats.get('seasonStats', {}), indent=2)[:1000])

print("\nCareer Stats (first 3 seasons):")
career = stats.get('careerStats', [])
print(json.dumps(career[:3], indent=2))

## 4. Get Draft Information

In [ ]:
# Get NHL draft info
draft_info = get_player_draft_info(player_id)

print("Draft Information:")
print(json.dumps(draft_info, indent=2))

## 5. Scrape Player Profile as DataFrame

In [ ]:
# Get profile as a pandas DataFrame (single row)
profile_df = scrape_player_profile(player_id)

print(f"Profile DataFrame shape: {profile_df.shape}")
print("\nColumns available:")
print(profile_df.columns.tolist())
print("\nProfile data:")
display(profile_df.T)  # Transpose for better viewing

## 6. Scrape Player Season Stats as DataFrame

In [ ]:
# Get season stats as DataFrame
stats_df = scrape_player_stats(player_id)

print(f"Stats DataFrame shape: {stats_df.shape}")
print("\nSeason statistics:")
display(stats_df.T)

## 7. Scrape Player Career Stats as DataFrame

In [ ]:
# Get career stats (multiple seasons) as DataFrame
career_df = scrape_player_career_stats(player_id)

print(f"Career stats shape: {career_df.shape}")
print("\nCareer statistics by season:")
display(career_df)

## 8. Scrape Player Game Log

In [ ]:
# Get game-by-game log as DataFrame
game_log_df = scrape_player_game_log(player_id)

print(f"Game log shape: {game_log_df.shape}")
print(f"Games played: {len(game_log_df)}")
print("\nFirst 10 games:")
display(game_log_df.head(10))

## 9. Scrape Player Shot Locations

In [ ]:
# Get shot location data as DataFrame
shots_df = scrape_player_shot_locations(player_id)

print(f"Shot locations shape: {shots_df.shape}")
print(f"Total shots tracked: {len(shots_df)}")
print("\nFirst 10 shots:")
display(shots_df.head(10))

## 10. Scrape Multiple Players

Get data for multiple players at once.

In [ ]:
# Example player IDs (replace with actual IDs)
player_ids = [10036, 10037, 10038]

# Get biographical data for multiple players
multi_players_df = scrape_multiple_players(player_ids, include_stats=True)

print(f"Multiple players shape: {multi_players_df.shape}")
print(f"Players scraped: {len(multi_players_df)}")
print("\nPlayer data:")
display(multi_players_df)

## 11. Analysis Example: Player Performance Trends

Analyze a player's performance across games.

In [ ]:
import matplotlib.pyplot as plt

# Get game log
game_log = scrape_player_game_log(player_id)

if not game_log.empty and 'points' in game_log.columns:
    # Calculate cumulative points
    game_log['cumulative_points'] = game_log['points'].cumsum()
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Points per game
    ax1.bar(range(len(game_log)), game_log['points'], alpha=0.7)
    ax1.set_title('Points Per Game')
    ax1.set_xlabel('Game Number')
    ax1.set_ylabel('Points')
    ax1.grid(True, alpha=0.3)
    
    # Cumulative points
    ax2.plot(game_log['cumulative_points'], marker='o', linewidth=2)
    ax2.set_title('Cumulative Points Over Season')
    ax2.set_xlabel('Game Number')
    ax2.set_ylabel('Total Points')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Stats summary
    print("\nSeason Summary:")
    print(f"Games Played: {len(game_log)}")
    print(f"Total Points: {game_log['points'].sum()}")
    print(f"Points Per Game: {game_log['points'].mean():.2f}")
else:
    print("No game log data available or 'points' column missing")

## 12. Analysis Example: Shot Location Visualization

Visualize where a player takes shots from.

In [ ]:
# Get shot locations
shots = scrape_player_shot_locations(player_id)

if not shots.empty and 'x' in shots.columns and 'y' in shots.columns:
    # Create shot chart
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Separate shots by outcome (if available)
    if 'outcome' in shots.columns:
        goals = shots[shots['outcome'] == 'goal']
        other_shots = shots[shots['outcome'] != 'goal']
        
        ax.scatter(other_shots['x'], other_shots['y'], 
                  c='blue', alpha=0.5, s=50, label='Shots')
        ax.scatter(goals['x'], goals['y'], 
                  c='red', alpha=0.8, s=100, marker='*', label='Goals')
    else:
        ax.scatter(shots['x'], shots['y'], 
                  c='blue', alpha=0.5, s=50, label='All Shots')
    
    ax.set_title('Shot Location Chart')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nTotal shots tracked: {len(shots)}")
    if 'outcome' in shots.columns:
        print(f"Goals: {len(shots[shots['outcome'] == 'goal'])}")
        print(f"Shooting %: {len(shots[shots['outcome'] == 'goal']) / len(shots) * 100:.1f}%")
else:
    print("No shot location data available")

## Summary

This notebook demonstrated:
1. Fetching raw player profile data
2. Getting biographical information
3. Retrieving season and career statistics
4. Getting draft information
5. Converting data to pandas DataFrames
6. Scraping game-by-game logs
7. Getting shot location data
8. Scraping multiple players at once
9. Analyzing player performance trends
10. Visualizing shot locations